In [1]:
# Network inspection

In [2]:
import copy, gtfs_functions
from tools.handlers.route_planner import Route_Planner
from tools.handlers.feed_handler import Feed_Handler
from tools.handlers.graph_handler import Graph_Handler

#feed_handler = Feed_Handler(input_file='data/gtfs_zips/oebb_gtfs.zip')
feed_handler = Feed_Handler(input_file='data/gtfs_zips/oebb_gtfs.zip')
graph_handler  = Graph_Handler()
G = graph_handler.read_graph('data/graphs/simple_graph_oebb.pickle')
route_planner = Route_Planner(feed=feed_handler.feed, G = G)

In [3]:
# Getting how many trains are stopping at given station
start_station = 'Salzburg Hauptbahnhof 1'
df = copy.deepcopy(feed_handler.feed.stops)
df['number_of_trips'] = df.apply(lambda r: feed_handler.get_number_of_trips_at_station(station_name=r['stop_name']) , axis=1)


INFO:root:Reading "stops.txt".
INFO:root:get trips in stop_times
INFO:root:accessing trips
INFO:root:Reading "routes.txt".
INFO:root:Reading "trips.txt".
INFO:root:Reading "trips.txt".
INFO:root:computing patterns
INFO:root:_trips is defined in stop_times
INFO:root:Reading "stop_times.txt".
INFO:root:Reading "stop_times.txt".


In [10]:
df

,stop_id,stop_name,stop_lat,stop_lon,zone_id,geometry,number_of_trips
0,at:41:3087:0:1,Bad Sauerbrunn Bahnhof,47.773870,16.324886,NaN,POINT (16.32489 47.77387),38
1,at:41:3087:0:2,Bad Sauerbrunn Bahnhof,47.773858,16.324859,NaN,POINT (16.32486 47.77386),38
2,at:41:3186:0:1,Breitenbrunn Bahnhof,47.941229,16.746385,NaN,POINT (16.74638 47.94123),35
3,at:41:3294:0:1,Deutschkreutz Bahnhof,47.603827,16.620054,NaN,POINT (16.62005 47.60383),10
4,at:41:3294:0:5,Deutschkreutz Bahnhof,47.603900,16.620028,NaN,POINT (16.62003 47.60390),10
...,...,...,...,...,...,...,...
2257,ua:22010,Chop,48.432706,22.205554,NaN,POINT (22.20555 48.43271),0
2258,ua:22025,Lvov,49.839852,23.994010,NaN,POINT (23.99401 49.83985),0
2259,ua:22033,Mukačevo,48.429189,22.711998,NaN,POINT (22.71200 48.42919),0
2260,ua:22072,Stryj,49.259615,23.841575,NaN,POINT (23.84158 49.25962),0


In [12]:
import folium

dot_size_variable = 'number_of_trips'

# Create a map object
m = folium.Map(location=[48, 16], zoom_start=7, clustered_marker=False)
cmap = folium.LinearColormap(
    ['#00FF00', '#FFFF00', '#FFA500', '#FF0000'],
    vmin=df[dot_size_variable].min(),
    vmax=df[dot_size_variable].max()
)

# Add markers for each station
for _, row in df.iterrows():
    tooltip = f"{row['stop_name']} - {int(row[dot_size_variable])} "        
    folium.CircleMarker(
        location=[row['stop_lat'], row['stop_lon']],
        radius=2,        
        tooltip=['{}: {} connections'.format(row['stop_name'], row[dot_size_variable])],
        fill=True,
        color=cmap(row[dot_size_variable]),
        fill_opacity=0.8,
        fill_color=cmap(row[dot_size_variable]),
        
        
    ).add_to(m)

# Show the map
m
